<a href="https://colab.research.google.com/github/redLoneWolf/TextSummarizer/blob/main/Text_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Text Summarizer using Huggingface Transformers library with Flask**

In [1]:
!git clone https://github.com/redLoneWolf/TextSummarizer.git

Cloning into 'TextSummarizer'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [13]:
%cd TextSummarizer

/content/TextSummarizer


In [14]:
!ls

main.py  README.md  requirements.txt  templates  Text_summarizer.ipynb


In [6]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!python main.py

^C
